# Testing Notebook
This notebook will be used for carrying out test cases as well as explaining/logicing the steps of the 2 phase simplex method employed in this repo. \
An understanding of the phase 1 simplex method is required to understand the following (as explaining/deriving everything would take a very long time).

In [ ]:
import random
import numpy as np

## Tableau Construction
Here we will justify and test how the tableau construction will work in our program.

Here our problems are solveable from the following form:\
$  \text{min }c\cdot x$ \
$  \text{s.t. } Ax = b$ \
$  x \geq 0$ 
\
\
Our auxllary problem (to solve problems with no apparent basis, i.e. no apparent identity basic matrix fromed in matrix A) is then: \
$  \text{min } e\cdot y$ \
$  \text{s.t. } Ax + I_n y = b$ \
$  x \geq 0, y \geq 0$ \
$ e = (1,...,1)^T$
\
\
Hence our problem for the auxillary problem can be proven to be the following: \
$\text{min } e\cdot(b-Ax)$ \
$  \text{s.t. } Ax + I_n y = b$ \
$  x \geq 0, y \geq 0$ \
$ e = (1,...,1)^T$ \
\
So our tableau becomes:
| $-e\cdot b$| $-eA$ | $0...0$ |
| :----------:| :-----:| :----: |
| $b$        |   $A$  | $I$ |

Notice the following facts:
1. Our tableau can be represented as a matrix of size $(m+1) \times (m+n+1)$
2. This tableau can be constructed numerically

We use numpy.linalg to compute matrix multiplication as prescribed above. \
Note that we only have a valid solution if the y variables are all 0, otherwise the original problem's constraints are violated.

### Tableau Construction for the 2 phase simplex method

### Tableau construction tests

In [ ]:
from two_phase_simplex import TwoPhaseSimplex

# test the basic tableau construction of the auxillary problem
# simple 2x2 matrix case
A = [[1,0],[1,1]]
b = [2,3]
c = [1,1]
x = TwoPhaseSimplex(A,b,c)
x.solve_phase_one()
print(x.get_tableau())

# non-square matrix
A = [[3,2,2],[4,5,4],[2,1,1],[0,1,1]]
b = [1,2,3,4]
c = [-1,-2,3]
y = TwoPhaseSimplex(A,b,c)
y.solve_phase_one()
print(y.get_tableau())

# check if these two outputs check out

### Tableau feasibility tests

#### Feasiblity via violation of auxillary problems non-negativity constaint

In [ ]:
# Simple problem (can be solved using normal gaussian elimination lol)

A = [[1,0],[1,1]]
b = [2,3]
c = [1,1]
x = TwoPhaseSimplex(A,b,c)
if (x.solve_program()):
    print(x.get_solution())

# unbounded problem

A = [[3,4,5],[42,5,1],[2,5,4]]
b = [1,2,3]
c = [2,3,1]

y = TwoPhaseSimplex(A,b,c)
if y.solve_program():
    print(y.get_solution())

# higher dimension bounded problem

A = [[2,3,1,0],[3,1,0,-1]]
b = [6,3]
c = [-4,-5,0,0]

z = TwoPhaseSimplex(A,b,c)
if z.solve_program():
    print(z.get_solution())

## Random Test Cases

In [ ]:
# produce random test cases

test_cases : dict = {"A" : [], "b" : [], "c" : []}

cases : int = 100

for i in range(cases):
    m : int = random.randint(2, 100)
    n : int = random.randint(2, 100)
    test_cases["A"].append(np.random.rand(m,n))
    test_cases["b"].append(np.random.rand(m))
    test_cases["c"].append(np.random.rand(n))

for i in range(cases):
    x = TwoPhaseSimplex(test_cases["A"][i], test_cases["b"][i], test_cases["c"][i])
    if x.solve_program():
        print(x.get_solution())

### Observations from the random tests
A very large amount of 'random' (generated by numpy) problems are infeasible or unbounded, hence do not have an optimal solution. In the next couple cells I will try to present these facts using graphs.

In [ ]:
def get_random_results(cases : np.array) -> np.array:
    """
    Runs the two phase simplex solver using random shape of matrices and entries.
    Returns a list of n results.
    """
    results = np.zeros(len(cases))
    for j,case in enumerate(cases):
        for i in range(int(case)):
            m : int = random.randint(9, 300)
            n : int = m - 5
            test_cases["A"].append(np.random.rand(m,n))
            test_cases["b"].append(np.random.rand(m))
            test_cases["c"].append(np.random.rand(n))


        for i in range(int(case)):
            x = TwoPhaseSimplex(test_cases["A"][i], test_cases["b"][i], test_cases["c"][i])
            if x.solve_program():
                results[j] += 1

    return results

In [ ]:
import matplotlib.pyplot as plt

x = np.logspace(1, 3, base=10)
print(x)
y = np.zeros(len(x))

y = get_random_results(x)

print(y)

## Input sanitisation tests

In [ ]:
# should raise exception with a message

A = [[3],[2],[4]] # A is a 3x1 matrix
b = [1,2] # b has size 2
c = [1] # c has size 1

x = TwoPhaseSimplex(A, b, c)

In [ ]:
# should raise exception with a message

A = [[3,-1],[2,5]] # A is a 3x2 matrix
b = [1,2,5] # b has size 3
c = [1] # c has size 1

x = TwoPhaseSimplex(A, b, c)

In [ ]:
# should work with no exception

A = [[3,-1],[2,5]] # A is a 3x2 matrix
b = [1,2] # b has size 2
c = [1,2] # c has size 2

x = TwoPhaseSimplex(A, b, c)